# Coursera Capstone Project for Data Science 




### Abstract: Using geospatial data leveraged from foursquare for the country Luxembourg (venues), Segmenting the country using Kmeans clustering so as to tell the right location for the openin of a new venue







After finding the data for the model, it is preprocessed so as to fit the objective

The dataframe will be be eventually reduced to a one hot encoded dataframe based on Venue Category before the model is trained. 
The rows of this dataframe are a 128 dimensioned flat vectors which are all zeroes except one position which is the category of that venue.



Once the model is obtained, we may pass a similar vector as the input to obtain the cluster label of the new entry

We may then check what latitude and longitude the cluster label signifies by comparing the dataframe which has the name of the state as well as Latitude and Longitude

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import requests
from sklearn.cluster import KMeans

In [2]:
%config Completer.use_jedi = False

In [3]:

df_places=pd.read_csv('luxmebourg_places.csv')

Selecting venues that are strictly within Luxembourg


In [4]:
venues_lex=df_places[df_places['location.country']=='Lëtzebuerg']
venues_lex.shape

(833, 18)

Cleaning the dataframe

In [5]:
lex_venues=venues_lex.drop(['location.country','location.crossStreet','location.address','location.formattedAddress','venuePage.id','location.neighborhood','Closest Town','location.cc','location.postalCode','id'],axis=1)

In [6]:
lex_venues.rename({'location.lat':'Latitude','location.lng':'Longitude','location.city':'City','location.state':'Canton'},axis=1,inplace=True)
lex_venues.head()

,index,name,Latitude,Longitude,City,Canton,Category,Category Type
0,0,Sport a Kulturzenter,49.887571,5.981206,Esch an der Sauer,Dikrech,Cultural Center,Professional & Other Places
1,485,Lëtzebuerg City Museum,49.610056,6.133499,Lëtzebuerg,Lëtzebuerg,Cultural Center,Professional & Other Places
2,1,Sports Direct,49.963722,5.857220,Pommerloch,Dikrech,Clothing Store,Shop & Service
4,878,Rebounce Shop North,49.847288,6.100420,Ettelbruck,Dikrech,Clothing Store,Shop & Service
6,2,Sporthal in de Bende,49.930961,6.035121,NaN,NaN,None,Professional & Other Places


In [7]:
a=lex_venues.dropna()
a.shape

(549, 8)

Ensuring the category of the venue is defined

In [8]:
b=a[a.Category!='None']

Defining a function that gets the states of the venues whose states are missing


In [ ]:
def getState(la,lo):
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            la, 
            lo, 
            radius, 
            LIMIT)
    results = requests.get(url).json()['response']['venues'][0]['location']['state']
    return results

Running a loop through the states columns to obtain states for missing values

In [ ]:
for i in range(0,len(states)):
    if type(states[i])==float:
        print(i)
        states[i]=getState(b['location.lat'][i],b['location.lng'][i])


one hot encoding the dataframe with respect to the Venue Category

In [9]:
lux_1hot = pd.get_dummies(b[['Category']])

In [10]:
lux_1hot.reset_index(inplace=True)

In [11]:
lux_1hot

,index,Category_Adult Education Center,Category_Afghan Restaurant,Category_Art Gallery,Category_Art Museum,Category_Asian Restaurant,Category_Athletics & Sports,Category_Auto Dealership,Category_BBQ Joint,Category_Bakery,...,Category_Steakhouse,Category_Student Center,Category_Sushi Restaurant,Category_Swim School,Category_Tailor Shop,Category_Tattoo Parlor,Category_Thai Restaurant,Category_University,Category_Wine Bar,Category_Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,63,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,960,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
520,963,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
521,964,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
522,965,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
lux_1hot.insert(0,'Canton',lex_venues['Canton'])


In [13]:
lux_1hot

,Canton,index,Category_Adult Education Center,Category_Afghan Restaurant,Category_Art Gallery,Category_Art Museum,Category_Asian Restaurant,Category_Athletics & Sports,Category_Auto Dealership,Category_BBQ Joint,...,Category_Steakhouse,Category_Student Center,Category_Sushi Restaurant,Category_Swim School,Category_Tailor Shop,Category_Tattoo Parlor,Category_Thai Restaurant,Category_University,Category_Wine Bar,Category_Wine Shop
0,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lëtzebuerg,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dikrech,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dikrech,63,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,NaN,960,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
520,Dikrech,963,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
521,Lëtzebuerg,964,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
522,NaN,965,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
lux_1hot.drop(['index'],axis=1,inplace=True)


In [15]:
lux_1hot

,Canton,Category_Adult Education Center,Category_Afghan Restaurant,Category_Art Gallery,Category_Art Museum,Category_Asian Restaurant,Category_Athletics & Sports,Category_Auto Dealership,Category_BBQ Joint,Category_Bakery,...,Category_Steakhouse,Category_Student Center,Category_Sushi Restaurant,Category_Swim School,Category_Tailor Shop,Category_Tattoo Parlor,Category_Thai Restaurant,Category_University,Category_Wine Bar,Category_Wine Shop
0,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lëtzebuerg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
520,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
521,Lëtzebuerg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
522,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
d=lux_1hot.dropna()
d

,Canton,Category_Adult Education Center,Category_Afghan Restaurant,Category_Art Gallery,Category_Art Museum,Category_Asian Restaurant,Category_Athletics & Sports,Category_Auto Dealership,Category_BBQ Joint,Category_Bakery,...,Category_Steakhouse,Category_Student Center,Category_Sushi Restaurant,Category_Swim School,Category_Tailor Shop,Category_Tattoo Parlor,Category_Thai Restaurant,Category_University,Category_Wine Bar,Category_Wine Shop
0,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lëtzebuerg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,Gréiwemaacher,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516,Lëtzebuerg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
517,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
520,Dikrech,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [17]:
venues_in_lux=d.reset_index()
venues_in_lux


,index,Canton,Category_Adult Education Center,Category_Afghan Restaurant,Category_Art Gallery,Category_Art Museum,Category_Asian Restaurant,Category_Athletics & Sports,Category_Auto Dealership,Category_BBQ Joint,...,Category_Steakhouse,Category_Student Center,Category_Sushi Restaurant,Category_Swim School,Category_Tailor Shop,Category_Tattoo Parlor,Category_Thai Restaurant,Category_University,Category_Wine Bar,Category_Wine Shop
0,0,Dikrech,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Lëtzebuerg,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,Dikrech,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Dikrech,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,Dikrech,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,515,Gréiwemaacher,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315,516,Lëtzebuerg,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
316,517,Dikrech,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
317,520,Dikrech,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Grouping by State/Canton and taking the mean to obtain what fraction of the venue type is in each state

In [18]:
g=venues_in_lux.groupby('Canton').mean()


In [19]:
g


,index,Category_Adult Education Center,Category_Afghan Restaurant,Category_Art Gallery,Category_Art Museum,Category_Asian Restaurant,Category_Athletics & Sports,Category_Auto Dealership,Category_BBQ Joint,Category_Bakery,...,Category_Steakhouse,Category_Student Center,Category_Sushi Restaurant,Category_Swim School,Category_Tailor Shop,Category_Tattoo Parlor,Category_Thai Restaurant,Category_University,Category_Wine Bar,Category_Wine Shop
Canton,,,,,,,,,,,,,,,,,,,,,
Berdorf,339.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
Dikrech,234.720930,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.011628,0.000000,0.023256,0.011628,0.0,0.000000,0.000000,0.0,0.000000,0.023256
Gréiwemaacher,284.769231,0.030769,0.000000,0.000000,0.000000,0.015385,0.000000,0.0,0.0,0.000000,...,0.000000,0.015385,0.000000,0.000000,0.0,0.030769,0.015385,0.0,0.000000,0.000000
Lëtzebuerg,261.078788,0.000000,0.006061,0.012121,0.006061,0.042424,0.018182,0.0,0.0,0.006061,...,0.000000,0.000000,0.006061,0.000000,0.0,0.000000,0.006061,0.0,0.006061,0.000000
Okres Luxemburg,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
Округ Дикирш,97.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [20]:
venues_by_state=g.drop('index',axis=1)
venues_by_state

,Category_Adult Education Center,Category_Afghan Restaurant,Category_Art Gallery,Category_Art Museum,Category_Asian Restaurant,Category_Athletics & Sports,Category_Auto Dealership,Category_BBQ Joint,Category_Bakery,Category_Bar,...,Category_Steakhouse,Category_Student Center,Category_Sushi Restaurant,Category_Swim School,Category_Tailor Shop,Category_Tattoo Parlor,Category_Thai Restaurant,Category_University,Category_Wine Bar,Category_Wine Shop
Canton,,,,,,,,,,,,,,,,,,,,,
Berdorf,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
Dikrech,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.011628,...,0.011628,0.000000,0.023256,0.011628,0.0,0.000000,0.000000,0.0,0.000000,0.023256
Gréiwemaacher,0.030769,0.000000,0.000000,0.000000,0.015385,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.015385,0.000000,0.000000,0.0,0.030769,0.015385,0.0,0.000000,0.000000
Lëtzebuerg,0.000000,0.006061,0.012121,0.006061,0.042424,0.018182,0.0,0.0,0.006061,0.012121,...,0.000000,0.000000,0.006061,0.000000,0.0,0.000000,0.006061,0.0,0.006061,0.000000
Okres Luxemburg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
Округ Дикирш,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


Setting up the K means Model using the venues dataframe

In [21]:
# set number of clusters
kclusters = 5

X = venues_in_lux.drop(['index','Canton'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_[0:10] 


Creating a Dataframe having most popular venues by State

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['State']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
state_venues_sorted = pd.DataFrame(columns=columns)
state_venues_sorted['State'] = venues_in_lux['Canton']

for ind in np.arange(venues_by_state.shape[0]):
    state_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_by_state.iloc[ind, :], num_top_venues)

state_venues_sorted.head()

,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dikrech,Category_Electronics Store,Category_Wine Shop,Category_Festival,Category_Coworking Space,Category_Cultural Center,Category_Daycare,Category_Deli / Bodega,Category_Department Store,Category_Design Studio,Category_Diner
1,Lëtzebuerg,Category_Restaurant,Category_Hotel,Category_French Restaurant,Category_Chinese Restaurant,Category_Portuguese Restaurant,Category_City Hall,Category_History Museum,Category_Playground,Category_Wine Shop,Category_Sushi Restaurant
2,Dikrech,Category_Electronics Store,Category_Hotel,Category_Parking,Category_Sporting Goods Shop,Category_Italian Restaurant,Category_Resort,Category_Museum,Category_Japanese Restaurant,Category_Pizza Place,Category_Shopping Mall
3,Dikrech,Category_Hotel,Category_Italian Restaurant,Category_Gym / Fitness Center,Category_Restaurant,Category_French Restaurant,Category_Asian Restaurant,Category_Parking,Category_College Gym,Category_Chinese Restaurant,Category_Building
4,Dikrech,Category_Hotel,Category_Wine Shop,Category_Festival,Category_Coworking Space,Category_Cultural Center,Category_Daycare,Category_Deli / Bodega,Category_Department Store,Category_Design Studio,Category_Diner


In [24]:
state_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

state_venues_sorted.head()

## Making a test variable to predict its cluster label

In [25]:
x=np.zeros((1,128))
x[0][70]=1
x

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [26]:
t=pd.DataFrame(x)


In [27]:
kmeans.predict(t)

array([0])

fin